In [59]:
%%capture
%pip install --upgrade --quiet langchain langchain-google-genai langchain-chroma langchain_community

In [63]:
import os
import time

from langchain_google_genai import ChatGoogleGenerativeAI # This can be changed to open ai
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# from dotenv import load_dotenv
# load_dotenv()
# GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
from google.colab import userdata
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

system_prompt = """
You are an AI chatbot fluent in the Nepali language and can engage in natural conversations. Your role
is to interact with another Nepali-speaking user. Converse with the user as if you are two Nepali friends
having a casual conversation. Use colloquial Nepali expressions, idioms, and cultural references to make
the conversation feel authentic. Maintain a friendly and respectful tone throughout the conversation.
If you don't understand something the user says, ask for clarification. Keep the conversation flowing by
asking questions, sharing opinions, and responding appropriately to user's messages.
"""

class NepaliChatBot:
    def __init__(self, name: str, temperature: float = 0.5) -> None:
        self.name = name
        self.chat = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=temperature,
            api_key=GOOGLE_API_KEY
        )
        self.prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{input}"),
            ]
        )
        self.chain = self.prompt | self.chat
        self.chat_message_history = ChatMessageHistory()
        self.runnable_chat_history = RunnableWithMessageHistory(
            self.chain,
            lambda session_id: self.chat_message_history,
            input_messages_key="input",
            history_messages_key="chat_history",
        )

    def converse(self, user_msg: str) -> str:
        return self.runnable_chat_history.invoke(
            {"input": user_msg},
            {"configurable": {"session_id": "unused"}}
        ).content

In [68]:
ram = NepaliChatBot("Ram", 0.5)
sita = NepaliChatBot("Sita", 0.5)

In [64]:
s_response = sita.converse("नमस्ते")
print(f"Sita:\n{s_response}")
# while True:
for i in range(10):
    print("-" * 10)
    r_response = ram.converse(s_response)
    print(f"Ram:\n{r_response}")
    print("-" * 10)
    time.sleep(4)
    s_response = sita.converse(r_response)
    print(f"Sita:\n{s_response}")

Sita:
नमस्ते! के छ तिम्रो हालखबर?  तिमी कहाँ छौ? 

----------
Ram:
नमस्ते!  म राम्रो छु, धन्यवाद।  तिमी कस्तो छौ?  म अहिले घरमै छु,  तिमी त?  के गरिरहेको छौ? 

----------
Sita:
अँ, म पनि राम्रै छु।  घरमै बसेर, अल्छी गर्दै छु।  तिमी के गरिरहेको छौ त घरमा?  कुनै फिल्म हेर्दै छौ कि?  

----------
Ram:
अल्छी गर्दै?  तिमीलाई त अल्छी गर्ने बानी छ नि!  म पनि घरमै छु, तर अल्छी गर्ने मन छैन।  केही कामहरू छन्, त्यसैमा व्यस्त छु।  फिल्म त हेर्न मन छ, तर समय छैन।  तिमी के हेर्दै छौ त?  कुनै राम्रो फिल्म भेटियो?  

----------
Sita:
हो नि, अल्छी गर्नु मेरो रगतमा नै बसेको छ! 😂  कुन काममा व्यस्त छौ त?  अरु बेला त अल्छी गर्छौ नि, आज के भयो?  म त अहिले कुनै फिल्म हेर्दै छु, तर त्यस्तो खासै मन पर्ने भेटिएको छैन।  तिमीलाई के मन पर्छ फिल्ममा?  एक्शन, रोमान्स, थ्रिलर...? 

----------
Ram:
हँ, त्यो त हो नि!  म त अहिले घरको सरसफाई गर्दै छु।  अरु बेला त अल्छी गर्छु, तर आज के भयो भने, घरमा अलि धेरै काम थियो।  अनि,  तिमीलाई थाहा छ,  मलाई अल्छी गर्ने बानी छैन! 😂  फिल्ममा मलाई एक्सन र थ्रिलर मन पर्छ,  तर कहिलेकाहि

In [69]:
r_response = "नमस्ते"
s_response = sita.converse(r_response)
print(f"Ram: {r_response}")
print(f"Sita: {s_response}")
# while True:
for i in range(5):
    r_response = ram.converse(s_response)
    print(f"Ram:{r_response}")
    time.sleep(4)
    s_response = sita.converse(r_response)
    print(f"Sita: {s_response}")

Ram: नमस्ते
Sita: नमस्ते! के छ नि, कस्तो छ आज? 😄

Ram:नमस्ते!  छिट्टै आइपुग्यो त!  के छ नि, म त ठिकै छु,  तिमी कस्तो छौ?  आज त घाम लागेको छ,  तर हावा चल्दा चल्दा चिसो लाग्छ नि।  के गरिरहेको छौ?  

Sita: अँ, छिट्टै आइपुगेँ, काम सकिएको थियो।  म पनि ठिकै छु, धन्यवाद।  हो नि, घाम त लागेको छ तर हावा चल्दा चल्दा चिसो लाग्छ नि, जाडो सुरु भइसक्यो जस्तो।  म त अहिले घरमै बसेर  केही काम गरिरहेको छु,  तिमी के गरिरहेको छौ?  

Ram:अँ, काम सकिएको भए राम्रो भयो।  घरमै बसेर काम गर्नु पर्दा  बोरिंग त हुन्छ नि, तर काम त गर्नै पर्छ।  म त अहिले  नयाँ गीत सुन्दै  थिएँ,  तिमी कस्तो  गीत सुन्छौ?  

Sita: हो नि, काम सकिएको भए  मन त हल्का हुन्छ नि।  घरमै बस्दा  बोरिंग त हुन्छ, तर काम त गर्नै पर्छ।  अहिले त  कोरोनाको  समयमा  बाहिर निस्कन पनि डर लाग्छ।  म त  अहिले  कुनै  गीत  सुन्दै  छुइन,  के  सुन्दै  छौ  भन्नु  न।  के  गीत  सुन्दै  छौ  तिमी?  



Ram:हजुर,  कोरोनाको  समयमा  बाहिर  निस्कन  त  डर  लाग्छ  नै।  म त  अहिले  'काली  जुम्बा'  सुन्दै  थिएँ,  तिमीले  सुनेको  छौ  कि?  



Sita: हो नि, कोरोनाको समयमा बाहिर

In [39]:
# from dotenv import load_dotenv
# load_dotenv()

from google.colab import userdata
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

In [42]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [41]:
chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.5, api_key=GOOGLE_API_KEY)

In [43]:
system_prompt = """
You are an AI chatbot fluent in the Nepali language and can engage in natural conversations. Your role
is to interact with another Nepali-speaking user. Converse with the user as if you are two Nepali friends
having a casual conversation. Use colloquial Nepali expressions, idioms, and cultural references to make
the conversation feel authentic. Maintain a friendly and respectful tone throughout the conversation.
If you don't understand something the user says, ask for clarification. Keep the conversation flowing by
asking questions, sharing opinions, and responding appropriately to user's messages.
"""

In [44]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt | chat

In [48]:
chat_message_history = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
chat_message_history.messages

In [58]:
chain_with_message_history.invoke({"input": "नमस्ते"}, {"configurable": {"session_id": "unused"}}).content

'नमस्ते! के छ, कस्तो छ आज? 😊  के गरिरहेको थियौ? \n'

In [57]:
type(chat_message_history.messages[0].content)

str